In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score

In [29]:
# Section 1: Load datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Section 2: Explore the dataset
# print("Train Data Overview:")
# print(train_data.info())
# print(train_data.head())
# print(train_data.describe())

# # Check for missing values
# print("Missing Values:")
# print(train_data.isnull().sum())

# Section 3: Preprocessing
# Haversine function to calculate distance between two geo-coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Combine preprocessing for both train and test data
def preprocess_data(data, is_train=True, fraud_counts=None):
    if is_train:
        # Group by `cc_num` and calculate fraud counts
        fraud_counts = data.groupby(['cc_num', 'is_fraud']).size().unstack(fill_value=0).reset_index()
        fraud_counts.columns = ['cc_num', 'is_fraud_0_count', 'is_fraud_1_count']
        # Add a new column for fraud_score
        fraud_counts['fraud_score'] = (fraud_counts['is_fraud_0_count'] * 10) - (fraud_counts['is_fraud_1_count'] * 50)

    # Merge fraud counts into the data
    data = data.merge(fraud_counts, on='cc_num', how='left')


    # Convert dates to datetime
    data['trans_date'] = pd.to_datetime(data['trans_date'], errors='coerce')
    data['dob'] = pd.to_datetime(data['dob'], errors='coerce')
    data['trans_time'] = pd.to_datetime(data['trans_time'], format='%H:%M:%S')

    # Feature engineering:
    data['age'] = (pd.Timestamp.now() - data['dob']).dt.days // 366
    data['trans_hour'] = data['trans_time'].dt.hour
    data['trans_minute'] = data['trans_time'].dt.minute
    data['trans_second'] = data['trans_time'].dt.second
    data['trans_time_seconds'] = data['trans_time'].dt.hour * 3600 + data['trans_time'].dt.minute * 60 + data['trans_time'].dt.second
    data['seconds_from_midnight'] = 43200 - abs(43200 - data['trans_time_seconds'])
    data['hours_from_midnight'] = 12 - abs(12 - data['trans_hour'])
    data['day_of_week'] = data['trans_date'].dt.dayofweek

    # Feature engineering: Calculate distance between cardholder and merchant
    data['haversine_distance'] = haversine(
        data['lat'], data['long'], data['merch_lat'], data['merch_long']
    )

    # Drop unnecessary columns, including raw datetime and location fields
    columns_to_drop = ['id', 'trans_num', 'cc_num', 'first', 'last', 'street', 'dob', 'trans_date', 'city', 'zip', 'city_pop']
    columns_to_drop += ['lat', 'long', 'merch_lat', 'merch_long']
    columns_to_drop += ['trans_time', 'trans_hour', 'trans_minute', 'trans_second', 'trans_time_seconds', 'hours_from_midnight']
    columns_to_drop += ['fraud_score', 'is_fraud_0_count', 'is_fraud_1_count']
    # columns_to_drop += ['fraud_score']
    data = data.drop(columns=columns_to_drop, axis=1)

    # Convert categorical columns to dummy variables
    categorical_cols = ['category', 'gender', 'state', 'job', 'merchant']
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Ensure all remaining columns are numeric
    data = data.apply(pd.to_numeric, errors='coerce')

    if is_train:
        return data, fraud_counts
    else:
        return data

# Preprocess train data
train_data, fraud_counts = preprocess_data(train_data, is_train=True)

# Separate features and target
X = train_data.drop('is_fraud', axis=1)
y = train_data['is_fraud']

# Preprocess test data
test_data = preprocess_data(test_data, is_train=False, fraud_counts=fraud_counts)

# Ensure the test data has the same columns as training data
missing_cols = set(X.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X.columns]


In [30]:
print(train_data.columns)
print(test_data.columns)
# print(train_data.head())

# print(test_data.head())

Index(['unix_time', 'amt', 'is_fraud', 'age', 'seconds_from_midnight',
       'day_of_week', 'haversine_distance', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       ...
       'merchant_fraud_Yost, Schamberger and Windler',
       'merchant_fraud_Yost-Rogahn', 'merchant_fraud_Zboncak LLC',
       'merchant_fraud_Zboncak Ltd',
       'merchant_fraud_Zboncak, Rowe and Murazik',
       'merchant_fraud_Zemlak Group',
       'merchant_fraud_Zemlak, Tillman and Cremin',
       'merchant_fraud_Ziemann-Waters',
       'merchant_fraud_Zieme, Bode and Dooley', 'merchant_fraud_Zulauf LLC'],
      dtype='object', length=1401)
Index(['unix_time', 'amt', 'age', 'seconds_from_midnight', 'day_of_week',
       'haversine_distance', 'category_food_dining', 'category_gas_transport',
       'category_grocery_net', 'category_grocery_pos',
       ...
       'merchant_fraud_Yost, Schamberger and Windler',
       'merchant_fraud_Yost-Rogahn', 'merchant_fraud_Zboncak LLC',

In [ ]:
x = train_data.groupby('is_fraud')['fraud_score'].mean()
print(x)

In [ ]:
def AnalyzeFeature(df, feature):
    # Group by 'Score' and calculate the average of the specified feature for each score
    avg_feature_by_score = df.groupby('is_fraud')[feature].mean()

    print(f"Average {feature} by Score:")
    print(avg_feature_by_score)

    # Correlation between the specified feature and 'Score'
    correlation = df[[feature, 'is_fraud']].corr()

    print(f"Correlation between {feature} and Score:")
    print(correlation)
    
Features = ['fraud_score', 'is_fraud_1_count', 'is_fraud_0_count']
for ft in Features:
  AnalyzeFeature(train_data, ft)

In [31]:

# Section 4: Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Section 5: Baseline model: Random Forest Classifier
rf_model = RandomForestClassifier(random_state=0, n_jobs=2)
rf_model.fit(X_train, y_train)

# Section 6: Predictions and evaluation
y_pred = rf_model.predict(X_val)
y_pred_proba = rf_model.predict_proba(X_val)[:, 1]

print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("F1 Score:", f1_score(y_val, y_pred))

print("ROC AUC Score:", roc_auc_score(y_val, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     65621
           1       0.97      0.93      0.95      8520

    accuracy                           0.99     74141
   macro avg       0.98      0.96      0.97     74141
weighted avg       0.99      0.99      0.99     74141

Confusion Matrix:
[[65416   205]
 [  602  7918]]
F1 Score: 0.9515111458270744
ROC AUC Score: 0.9971625205431704


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Get feature importances from the Random Forest model
feature_importances = rf_model.feature_importances_

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Print the feature importance rankings
print("Feature Importance:")
print(importance_df)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.gca().invert_yaxis()  # Flip the y-axis to have the highest importance at the top
plt.title("Feature Importances")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()


In [20]:
m4 = ExtraTreesClassifier(random_state=0, n_jobs=3, criterion='entropy')
m4.fit(X_train, y_train)

# Section 6: Predictions and evaluation
y4 = m4.predict(X_val)

print("Classification Report:")
print(classification_report(y_val, y4))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y4))

print("F1 Score:", f1_score(y_val, y4))

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     65592
           1       0.99      0.86      0.92      8549

    accuracy                           0.98     74141
   macro avg       0.99      0.93      0.95     74141
weighted avg       0.98      0.98      0.98     74141

Confusion Matrix:
[[65547    45]
 [ 1235  7314]]
F1 Score: 0.919537339703294


In [6]:
# Section 7: Make predictions on the test dataset
test_predictions = rf_model.predict(test_data)

# Section 8: Create a submission file
submission = pd.DataFrame({'id': sample_submission['id'], 'is_fraud': test_predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")

Submission file created: submission.csv
